# PSD Analysis Example

This notebook demonstrates a complete PSD (Pulse Shape Discrimination) analysis workflow
for neutron/gamma discrimination using the psd_analysis toolkit.

## 1. Setup and Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add parent directory to path if running from notebooks/
sys.path.insert(0, '..')

from psd_analysis import (
    load_psd_data,
    validate_events,
    calculate_psd_ratio,
    calibrate_energy,
    calculate_figure_of_merit
)
from psd_analysis.visualization import (
    plot_psd_scatter,
    plot_energy_spectra,
    plot_calibration_curve
)

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 2. Load Data

Load raw waveform data from CSV files. Update the file paths to point to your data.

In [ ]:
# Generate synthetic data for demonstration
import numpy as np
import pandas as pd

print("Generating synthetic demonstration data...")

# Create synthetic PSD data
n_events = 1000
np.random.seed(42)

# Simulate gamma and neutron events
gamma_events = pd.DataFrame({
    'ENERGY': np.random.uniform(100, 2000, n_events//2),
    'ENERGYSHORT': np.random.uniform(70, 1400, n_events//2),
    'PARTICLE': 'gamma'
})

neutron_events = pd.DataFrame({
    'ENERGY': np.random.uniform(100, 2000, n_events//2),
    'ENERGYSHORT': np.random.uniform(40, 1000, n_events//2),  # More tail
    'PARTICLE': 'neutron'
})

df = pd.concat([gamma_events, neutron_events], ignore_index=True)

print(f"Generated {len(df)} events")
print(f"Columns: {df.columns.tolist()}")

# For real data, use:
# df = load_psd_data('../data/raw/your_measurement.csv')

## 3. Quality Control

Apply quality control filters to remove saturated pulses, pile-up events, and unstable baselines.

In [ ]:
# For synthetic data, we'll skip QC since it's clean
# For real data, use:
# valid_mask, qc_report = validate_events(df)
# df_clean = df[valid_mask].copy()

df_clean = df.copy()
print(f"Data ready: {len(df_clean)} events")

## 4. Energy Calibration

Calibrate energy scale using known gamma sources (Cs-137, Co-60, etc.).

In [ ]:
# Define calibration points (ADC channel, energy in keV)
# Example: Cs-137 Compton edge and photopeak
# calibration_points = [
#     (2500, 477.0),   # Cs-137 Compton edge
#     (3500, 661.7)    # Cs-137 photopeak
# ]

# Apply calibration
# df_clean = calibrate_energy(
#     df_clean,
#     calibration_points,
#     method='linear'
# )

# Plot calibration curve
# plot_calibration_curve(calibration_points, df_clean['cal_func'].iloc[0])
# plt.title('Energy Calibration')
# plt.show()

## 5. PSD Analysis

Calculate PSD parameter for neutron/gamma discrimination.

In [ ]:
# Calculate PSD ratio
df_clean = calculate_psd_ratio(df_clean)

print("\nPSD Statistics:")
print(f"  Mean PSD: {df_clean['PSD'].mean():.4f}")
print(f"  Std PSD: {df_clean['PSD'].std():.4f}")
print(f"  PSD range: [{df_clean['PSD'].min():.4f}, {df_clean['PSD'].max():.4f}]")

# Check by particle type
for particle in ['gamma', 'neutron']:
    particle_data = df_clean[df_clean['PARTICLE'] == particle]
    print(f"  {particle.capitalize()}: mean PSD = {particle_data['PSD'].mean():.4f}")

## 6. Visualization

Create PSD scatter plots and energy spectra.

In [ ]:
# PSD scatter plot
fig, ax = plt.subplots(figsize=(10, 6))
plot_psd_scatter(df_clean, ax=ax)
plt.title('PSD Scatter Plot - Synthetic Data')
plt.show()

In [ ]:
# Energy spectrum
# fig, ax = plt.subplots(figsize=(10, 6))
# plot_energy_spectra(
#     df_clean,
#     separate_particles=True,  # Show neutron/gamma separately if labeled
#     ax=ax
# )
# plt.show()

## 7. Figure of Merit

Calculate FoM for PSD performance (requires labeled data).

In [ ]:
# Calculate Figure of Merit with labeled data
neutron_psd = df_clean[df_clean['PARTICLE'] == 'neutron']['PSD'].values
gamma_psd = df_clean[df_clean['PARTICLE'] == 'gamma']['PSD'].values

fom = calculate_figure_of_merit(neutron_psd, gamma_psd)

## 8. Machine Learning Classification

Optional: Use ML models for enhanced discrimination.

In [ ]:
# from psd_analysis.ml.classical import ClassicalMLClassifier

# # Load pre-trained model
# clf = ClassicalMLClassifier(method='random_forest')
# clf.load('../models/psd_classifier.pkl')

# # Predict
# predictions, probabilities = clf.predict(df_clean)
# df_clean['PARTICLE_ML'] = ['neutron' if p==1 else 'gamma' for p in predictions]
# df_clean['CONFIDENCE'] = probabilities.max(axis=1)

# print(f"\nML Classification:")
# print(f"  Neutrons: {(predictions==1).sum()}")
# print(f"  Gammas: {(predictions==0).sum()}")
# print(f"  Mean confidence: {df_clean['CONFIDENCE'].mean():.3f}")

## 9. Isotope Identification

Identify isotopes from gamma spectrum.

In [ ]:
# from psd_analysis.spectroscopy import identify_isotopes

# # Identify isotopes (requires calibrated energy)
# if 'ENERGY_KEV' in df_clean.columns:
#     # Select gamma events if available
#     if 'PARTICLE' in df_clean.columns:
#         df_gamma = df_clean[df_clean['PARTICLE'] == 'gamma']
#     else:
#         df_gamma = df_clean
#     
#     results = identify_isotopes(
#         df_gamma['ENERGY_KEV'].values,
#         prominence=50,
#         tolerance_keV=10,
#         min_confidence=0.5
#     )
#     
#     print("\nIdentified Isotopes:")
#     for isotope, info in results['identified_isotopes'].items():
#         print(f"  {isotope}: confidence={info['confidence_score']:.3f}")
#         print(f"    Energies: {info['matched_energies']} keV")

## 10. Export Results

In [ ]:
# Save processed data
# df_clean.to_csv('../results/processed/analyzed_data.csv', index=False)
# print("Results saved to ../results/processed/analyzed_data.csv")

## Next Steps

- Adjust energy calibration for your specific detector
- Train ML models on your labeled data using `scripts/ml_train.py`
- Characterize your scintillator using `scripts/scintillator_characterization.py`
- Generate comprehensive reports using `scripts/generate_reports.py`

For more information, see the [README](../README.md) and [documentation](../docs/).